# Transferência CSV -> SGBD

## Carregando CSV

In [54]:
import os
import psycopg2
import pandas as pd

df = pd.read_csv('../database/credit_card_transactions.csv')
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0


## Conexão com o PostrgreSQL

In [55]:
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

def get_connection():
    return psycopg2.connect(database="transactions",
                        user=DB_USER,
                        password=DB_PASSWORD,
                        host=DB_HOST, port=DB_PORT) 


## Inserindo dados de cidades

In [ ]:
conn = get_connection()
cur = conn.cursor()

df = df.rename(columns={'city': 'city_name', 'state': 'city_state'})
data = df[['city_name', 'city_state', 'city_pop']].values.tolist()


insert_query = """
    INSERT INTO public.city (city_name, city_state, city_pop)
    VALUES (%s, %s, %s)
"""

cur.executemany(insert_query, data)

conn.commit()
cur.close() 
conn.close() 

## Inserindo dados de clientes

In [ ]:
conn = get_connection()
cur = conn.cursor()

df['dob'] = pd.to_datetime(df['dob']).dt.date 
df_unique = df.drop_duplicates(subset=['first', 'last'])

city_query = """
    SELECT city_id FROM city WHERE city_name = %s AND city_state = %s
"""

city_ids = []
for _, row in df_unique.iterrows():
    cur.execute(city_query, (row['city'], row['state']))
    city_id = cur.fetchone()
    if city_id:
        city_ids.append(city_id[0])
    else:
        city_ids.append(None)

df_unique['city_id'] = city_ids

data_to_insert = df_unique[['first', 'last', 'gender', 'street', 'zip', 'lat', 'long', 'job', 'dob', 'city_id']].values.tolist()

insert_query = """
    INSERT INTO client (
        client_first_name, client_last_name, client_gender, client_street,
        client_zip, client_lat, client_long, client_job,
        client_date_of_birth, city_id
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)

conn.commit()
cur.close() 
conn.close() 

## Inserindo dados de cartões de crédito

In [ ]:
conn = get_connection()
cur = conn.cursor()

df_unique = df.drop_duplicates(subset=['cc_num'])

client_query = """
    SELECT client_id FROM client WHERE client_first_name = %s AND client_last_name = %s
"""

client_ids = []
for _, row in df_unique.iterrows():
    cur.execute(client_query, (row['first'], row['last']))
    client_id = cur.fetchone()
    if client_id:
        client_ids.append(client_id[0])
    else:
        client_ids.append(None)

df_unique['client_id'] = client_ids
data_to_insert = df_unique[['cc_num', 'client_id']].values.tolist()

insert_query = """
    INSERT INTO credit_card (cc_number, client_id)
    VALUES (%s, %s)
"""

cur.executemany(insert_query, data_to_insert)

conn.commit()
cur.close() 
conn.close() 

## Inserindo  dados de mercadores

In [ ]:
conn = get_connection()
cur = conn.cursor()

df['merch_zipcode'] = df['merch_zipcode'].astype(str).replace('nan', None)
df_unique = df.drop_duplicates(subset=['merchant'])
data_to_insert = df_unique[['merchant', 'merch_lat', 'merch_long', 'merch_zipcode']].values.tolist()

insert_query = """
    INSERT INTO merchant (merchant, merch_lat, merch_long, merch_zipcode)
    VALUES (%s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)

conn.commit()
cur.close()
conn.close()

## Inserindo dados de transações

In [ ]:
conn = get_connection()
cur = conn.cursor()

df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['is_fraud'] = df['is_fraud'].astype(bool)

cc_query = "SELECT cc_number FROM credit_card WHERE cc_number = %s"
merchant_query = "SELECT merchant_id FROM merchant WHERE merchant = %s"

cc_nums = []
merchant_ids = []

for _, row in df.iterrows():
    cur.execute(cc_query, (row['cc_num'],))
    cc = cur.fetchone()
    cc_nums.append(cc[0] if cc else None)
    
    cur.execute(merchant_query, (row['merchant'],))
    merchant = cur.fetchone()
    merchant_ids.append(merchant[0] if merchant else None)

df['cc_num_db'] = cc_nums
df['merchant_id_db'] = merchant_ids

data_to_insert = df[[
    'trans_num', 'category', 'amt', 
    'trans_date_trans_time', 'unix_time', 
    'is_fraud', 'cc_num_db', 'merchant_id_db'
]].values.tolist()

insert_query = """
    INSERT INTO transaction (
        trans_num, trans_category, trans_amt, trans_date_trans_time, 
        trans_unix_time, trans_is_fraud, cc_num, merchant_id
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)

conn.commit()
cur.close()
conn.close()